# 3. Feature Enginerring

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
import os 
import sys
sys.path.append(os.path.join(os.path.abspath('..')))

In [3]:
import src.data_loading as dl
import scripts.feature_enigneering as fe

In [4]:
output_data = '../data/processed'
df = pd.read_csv(output_data + '/processed_data.csv')
df.head()

,TransactionId,BatchId,AccountId,SubscriptionId,CustomerId,CurrencyCode,CountryCode,ProviderId,ProductId,ProductCategory,ChannelId,Amount,Value,TransactionStartTime,PricingStrategy,FraudResult
0,76871,36123,3957,887,4406,UGX,256,6,10,airtime,3,1000.0,1000,2018-11-15T02:18:49Z,2,0
1,73770,15642,4841,3829,4406,UGX,256,4,6,financial_services,2,-20.0,20,2018-11-15T02:19:08Z,2,0
2,26203,53941,4229,222,4683,UGX,256,6,1,airtime,3,500.0,500,2018-11-15T02:44:21Z,2,0
3,380,102363,648,2185,988,UGX,256,1,21,utility_bill,3,20000.0,21800,2018-11-15T03:32:55Z,2,0
4,28195,38780,4841,3829,988,UGX,256,4,6,financial_services,2,-644.0,644,2018-11-15T03:34:21Z,2,0


1. Create Aggregate Features

In [5]:
# Total transaction amount 
df['TotalTransactionAmount'] = df.groupby('CustomerId')['Value'].transform('sum')

In [6]:
# df['TotalT'] = df['CustomerId'].map(df.groupby('CustomerId')['Amount'].apply(lambda x: x.abs().sum()))

In [7]:
# Average Transaction Amount
df['AverageTransactionAmount'] = df['CustomerId'].map(df.groupby('CustomerId')['Amount'].apply(lambda x: x.abs().mean()))

In [8]:
# df['ava'] = df.groupby('CustomerId')['Value'].transform('mean')

In [9]:
# Transaction Count
df['TransationCount'] = df.groupby('CustomerId')['TransactionId'].transform('size')

Note that the std function will calculate the population standard deviation by default. So i want to calculate the sample standard deviation, so i can use the ddof parameter:

In [10]:
# Standard Deviation of Transaction Amounts
df['StdTransactionAmount'] = df['CustomerId'].map(df.groupby('CustomerId')['Amount'].apply(lambda x: x.abs().std(ddof=1)))

In [11]:
# df['StdDevTransaction'] = df.groupby('CustomerId')['Value'].transform('std')

In [12]:
#categorical and numerical columns into variables
num_col = df.select_dtypes(include=np.number).columns
cat_col = df.select_dtypes(exclude=np.number).columns

## 2. Extract Features

In [13]:
# Apply the function
fe.extract_features(df, ['TransactionStartTime'])

In [14]:
pd.set_option('display.max_columns', None)
df.sample(6)

,TransactionId,BatchId,AccountId,SubscriptionId,CustomerId,CurrencyCode,CountryCode,ProviderId,ProductId,ProductCategory,ChannelId,Amount,Value,TransactionStartTime,PricingStrategy,FraudResult,TotalTransactionAmount,AverageTransactionAmount,TransationCount,StdTransactionAmount,TransactionStartTime_Month,TransactionStartTime_Week,TransactionStartTime_Day,TransactionStartTime_Dayofweek,TransactionStartTime_Dayofyear,TransactionStartTime_Is_month_end,TransactionStartTime_Is_month_start,TransactionStartTime_Is_quarter_end,TransactionStartTime_Is_quarter_start,TransactionStartTime_Is_year_end,TransactionStartTime_Is_year_start,TransactionStartTime_Hour,TransactionStartTime_Minute,TransactionStartTime_Second,TransactionStartTime_Elapsed
87327,54076,39620,4670,3642,5138,UGX,256,6,3,airtime,3,2000.0,2000,2019-02-06 19:01:07,2,0,2257225,7270.746753,308,1.240273e+04,2,6,6,2,37,False,False,False,False,False,False,19,1,7,1549479667
79407,64702,45452,572,2950,909,UGX,256,5,15,financial_services,3,230000.0,230000,2019-01-31 14:52:21,2,0,83480760,725919.652174,115,1.297134e+06,1,5,31,3,31,True,False,False,False,False,False,14,52,21,1548946341
89401,20310,66243,4840,3829,5196,UGX,256,4,6,financial_services,2,-1000.0,1000,2019-02-08 09:47:52,2,0,32000,5333.333333,6,3.945462e+03,2,6,8,4,39,False,False,False,False,False,False,9,47,52,1549619272
48502,35278,9251,318,3087,647,UGX,256,6,3,airtime,3,1000.0,1000,2019-01-01 13:25:01,2,0,3905264,2076.180845,1869,7.680888e+03,1,1,1,1,1,False,True,False,True,False,True,13,25,1,1546349101
16431,98447,137393,4249,4429,7343,UGX,256,4,3,airtime,2,-25000.0,25000,2018-12-04 16:54:15,4,0,104900000,25641.652408,4091,1.295260e+04,12,49,4,1,338,False,False,False,False,False,False,16,54,15,1543942455
10945,67369,22573,4841,3829,3882,UGX,256,4,6,financial_services,2,-26.0,26,2018-11-30 04:19:32,2,0,446922,1655.865672,268,4.404733e+03,11,48,30,4,334,True,False,False,False,False,False,4,19,32,1543551572


In [15]:
# Boolean to int 
df[df.select_dtypes(include='bool').columns] = df.select_dtypes(include='bool').astype(int)

## Encode Categorical Variables

In [16]:
# Frequency Encoding
fe.encode_FE(df, ['CustomerId', 'TransactionId'])

CustomerId_FE , TransactionId_FE , 

In [22]:
# Label Encoding
fe.encode_LE(df, ['ProductCategory', 'ProductId', 'SubscriptionId', 'CurrencyCode', 'CountryCode', 'ProviderId', 'ChannelId', 'PricingStrategy'])

ProductCategory , ProductId , SubscriptionId , CurrencyCode , CountryCode , ProviderId , ChannelId , PricingStrategy , 

In [23]:
# Group Aggregation
fe.encode_AG(df, 'CustomerId', ['Value'], ['mean','min','max','std'])

CustomerId_Value_mean , CustomerId_Value_min , CustomerId_Value_max , CustomerId_Value_std , 

In [19]:
# Combine Features
# fe.encode_CB(df, 'ProductId', 'ProductCategory')

In [20]:
# Group Aggregation NUnique
# fe.encode_AG2(df, 'CustomerId', ['TransactionId'])